<a href="https://colab.research.google.com/github/kaushiksrini/cs482-finalproject/blob/master/cs482_finalproject_4mer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import sklearn.model_selection
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
"""file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))"""

nbp = drive.CreateFile({'id': '1H_pbNqLMrLqisLwqDP4SNF5AejrBw6XX'})
nbp.GetContentFile('nbp.csv')
bp = drive.CreateFile({'id': '1ENXkJyRRy45TK4SntiGrmxY1jF6qTx-r'})
bp.GetContentFile('bp.csv')

In [82]:
def char_to_val(char):
  if char == 'a':
    return 0
  if char == 't':
    return 1
  if char == 'c':
    return 2
  if char == 'g':
    return 3
  raise UnexpectedNucleotideError(char)

"""with open('nbp.csv', 'r') as f:
  nbp = list(csv.reader(f, delimiter=','))
with open('bp.csv', 'r') as f:
  bp = list(csv.reader(f, delimiter=','))"""

#nbp = [nbp[i][0] + nbp[i][1] for i in range(len(nbp))]
#bp = [bp[i][0] + nbp[i][1] for i in range(len(bp))]

def deconstruct(data):
  for i in range(len(data)):
    data[i] = [char_to_val(x) for x in data[i]]

#deconstruct(nbp)
#deconstruct(bp)

  
df_nbp = pd.read_csv('nbp.csv', header=None)
print(df_nbp.shape)
df_bp = pd.read_csv('bp.csv', header=None)
print(df_bp.shape)

(48643, 102)
(2699, 102)


In [83]:
def to_categorical(df):
  for col in df:
    df[col] = df[col].astype('category')

df_nbp['Brain'] = 1
df_bp['Brain'] = 0
df_total = df_nbp.append(df_bp)

y = df_total['Brain']
df_total.drop(['Brain'], axis=1, inplace=True)
x = df_total

to_categorical(x)
x = pd.get_dummies(x)

print(x.shape)
print(y.shape)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(51342, 25825)
(51342,)
(41073, 25825)
(10269, 25825)
(41073,)
(10269,)


In [84]:
y_test.head(100)

20413    1
3485     1
37016    1
1613     1
27498    1
8942     1
32391    1
24051    1
11468    1
2367     0
27215    1
40783    1
7081     1
12745    1
48348    1
39418    1
9945     1
14067    1
2714     1
23138    1
46872    1
40192    1
27077    1
481      0
11765    1
24824    1
24374    1
14764    1
40627    1
28607    1
        ..
17835    1
10371    1
18849    1
6205     1
22418    1
36694    1
32184    1
13253    1
31856    1
18032    1
14726    1
44393    1
20104    1
3174     1
4283     1
11379    1
37453    1
18410    1
9199     1
1297     0
46739    1
1834     0
18230    1
48220    1
34259    1
26510    1
47599    1
31396    1
5739     1
18893    1
Name: Brain, Length: 100, dtype: int64

In [0]:
from keras import optimizers
from keras import models
from keras import layers
from keras import metrics
from keras.layers import Dense, Activation

In [0]:
model = models.Sequential()
model.add(Dense(100, activation='relu', input_dim=25825))
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [88]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Train on 41073 samples, validate on 10269 samples
Epoch 1/10
41073/41073 [==============================] - 39s 948us/step - loss: 0.0511 - acc: 0.9859 - val_loss: 0.0229 - val_acc: 0.9941
Epoch 2/10
41073/41073 [==============================] - 38s 937us/step - loss: 0.0158 - acc: 0.9962 - val_loss: 0.0231 - val_acc: 0.9949
Epoch 3/10
41073/41073 [==============================] - 39s 938us/step - loss: 0.0117 - acc: 0.9978 - val_loss: 0.0189 - val_acc: 0.9961
Epoch 4/10
41073/41073 [==============================] - 39s 940us/step - loss: 0.0104 - acc: 0.9982 - val_loss: 0.0301 - val_acc: 0.9959
Epoch 5/10
41073/41073 [==============================] - 39s 944us/step - loss: 0.0101 - acc: 0.9985 - val_loss: 0.0190 - val_acc: 0.9966
Epoch 6/10
41073/41073 [==============================] - 39s 942us/step - loss: 0.0101 - acc: 0.9987 - val_loss: 0.0215 - val_acc: 0.9968
Epoch 7/10
41073/41073 [==============================] - 38s 923us/step - loss: 0.0098 - acc: 0.9987 - val_loss: 0.

In [89]:
y_preds = model.predict(x_test)
print(y_preds)

[[9.9985802e-01]
 [9.9998879e-01]
 [9.9953973e-01]
 ...
 [4.3611054e-10]
 [9.9920279e-01]
 [9.9992931e-01]]
